# AutoGPT with LangChain implementation

Good Practices
- Execute one task at a time. Each task should have verifiable input and output, so that if null or unexpected results, we can automatically retry the task. Each task can have multiple steps that are low-risk of failure. 


Issues
- Sometimes chain finishes too early. 
- Repeated online searches. 
- Logging output seems duplicated. 

In [1]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        func=search.run,
        description="useful for when you need to answer questions about current events through Internet. You should ask targeted questions",
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [2]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [3]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

In [4]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

In [5]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Tom",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=vectorstore.as_retriever(),
)
# Set verbose to be true
agent.chain.verbose = True

In [6]:
# agent.run(["read the WSJ article in data/nio_stumbles.txt. Write a summary. "])


In [7]:
# agent.run([
#     "Read the article summary in data/nio_summary.txt.", 
#     "Come up with 10 intersting questions to ask. ",
#     "Write the questions to data/nio_questions.txt in form of numbered lists. ",
# ])


In [8]:
agent.run([
    "Read the questions in data/nio_questions.txt. Choose one question that is most thought-provoking.", 
    "Search online to find answers to the question. Compile a list of evidence for support.",
    "Write the answer and evidence to data/nio_answer.txt."
])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are Tom, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Read the questions in data/nio_questions.txt. Choose one question that is most thought-provoking.
2. Search online to find answers to the question. Compile a list of evidence for support. Include actual URL to the website.
3. Write the answer and evidence to data/nio_answer.txt.


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "c

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


KeyboardInterrupt: 

In [13]:
from langchain.schema import (
    AIMessage,
    BaseMessage,
    Document,
    HumanMessage,
    SystemMessage,
)

In [18]:
# print out all actions and results
system_messages = [m for m in agent.full_message_history if isinstance(m, SystemMessage)]
for m in system_messages:
    print(m.content[:100].replace("\n", " "))

Command read_file returned: 1. What are the challenges faced by NIO in the Chinese EV market? 2. How
Command read_file returned: 1. What are the challenges faced by NIO in the Chinese EV market? 2. How
Command search returned: One of China's most vaunted EV startups, NIO is now a symbol of the challen
Command write_file returned: File written successfully to data/nio_answer.txt.
